In [1]:
import numpy as np
import pandas as pd
import os
from os.path import join
from glob import glob

from scipy.interpolate import interp1d
from scipy.io import loadmat
from scipy.signal import filtfilt, butter, hilbert
from skimage.measure import find_contours
from skimage.draw import polygon2mask

import matplotlib.pyplot as plt
from common.preprocess import get_occupancy, placefield, complete_contourline, segment_fields, identify_pairs



In [355]:
# Load all files into a dataframe (per recording session)
spikes_dir = 'data/emankin/principal_cells_and_path'
lfp_dir = 'data/emankin/lfp'

df_dict = dict(
    animal=[],
    date=[],
    ctxid=[],
    region=[],
    indata=[],
    spikedata=[],
    lfp_pth=[],
)
for walk in os.walk(spikes_dir):
    if len(walk[2]) > 0:
        dir_path_split = walk[0].split('/')
        rat_num, date = dir_path_split[3], dir_path_split[4]
        if (rat_num == 'rat624') and (date == '2013-09-19'):
            print('Broken LFP file, skip')
            continue
        region_indata = dict()
        region_spikedata = dict()
        region_cellinfo = dict()
        region_ctxid = dict()
        for filename in walk[2]:
            filepath = os.path.join(walk[0], filename)
            name_base, ext = os.path.splitext(filename)
            datakind, day_num, region = name_base.split('_')

            if datakind == 'spikeData':
                spikedata = loadmat(filepath)
                region_spikedata[region] = spikedata
            elif datakind == 'indata':
                indata = loadmat(filepath)
                region_indata[region] = indata
            elif datakind == 'ctxID':
                ctxid = loadmat(filepath)
                region_ctxid[region] = ctxid
                
            # Corresponding lfp file
            lfp_filedir = os.path.split(filepath)[0].replace('principal_cells_and_path', 'lfp')
            lfp_pth = join(lfp_filedir, 'rawLFP__Full.mat')
                
        assert sorted(region_spikedata.keys()) == sorted(region_indata.keys())
        all_regions_found = sorted(region_spikedata.keys())
        for region_found in all_regions_found:
                df_dict['animal'].append(rat_num)
                df_dict['date'].append(date)
                df_dict['region'].append(region_found)
                df_dict['indata'].append(region_indata[region_found])
                df_dict['spikedata'].append(region_spikedata[region_found])
                df_dict['ctxid'].append(region_ctxid[region_found])
                df_dict['lfp_pth'].append(lfp_pth)
df_temp = pd.DataFrame(df_dict)

Broken LFP file, skip


In [356]:
# Expand sessions to trials
# Find max lfp
inref_idx = dict(x=0, y=1, t=2, angle=3, v=4)
spref_idx = dict(tsp=4, xsp=5, ysp=6, vsp=7, anglesp=10)
spike_df_cols = ['tsp', 'xsp', 'ysp', 'vsp', 'anglesp']
indata_df_cols = ['x', 'y', 't', 'angle', 'v']
lfpref_idx = dict(t=0, lfp=1, fs=2)

df_dict = dict(
    animal=[],
    date=[],
    behavior=[],
    trial=[],
    region=[],
    position=[],
    spikes=[],
    wave=[]
)

for i in range(df_temp.shape[0]):
    print('\r %d/%d' % (i, df_temp.shape[0]), end='', flush=True)
    animal, date, region, ctxid, indata, spikedata = df_temp.iloc[i][['animal', 'date', 'region', 'ctxid', 'indata', 'spikedata']]
    lfp_pth = df_temp.iloc[i]['lfp_pth']
    lfp = loadmat(lfp_pth)['rawLFPData']
    
    
    tsp, xsp, ysp, vsp, anglesp = [spikedata['spikeData'][0][0][spref_idx[x]] for x in spike_df_cols]
    ntrial_tsp = tsp.shape[0]
    ntrial_indata = indata['indata'].shape[0]
    assert ntrial_indata == ntrial_tsp
    
    spike_df_dict = dict()
    indata_df_dict = dict()
    for trial_idx in range(ntrial_tsp):
        
        # Spike data
        spike_df_dict['tsp'] = [tsp[trial_idx, x].squeeze() for x in range(tsp[trial_idx, ].shape[0]) ]
        spike_df = pd.DataFrame(spike_df_dict)
        
        # Indata
        x, y, t, angle, v = [indata['indata'][trial_idx][0][inref_idx[x]] for x in indata_df_cols]
        indata_df_dict['t'] = list(t.squeeze())
        indata_df_dict['x'] = list(x.squeeze())
        indata_df_dict['y'] = list(y.squeeze())
        indata_df = pd.DataFrame(indata_df_dict)
        
        # Wave
        num_chs = lfp.shape[1]
        thetadeltaratio = np.zeros(num_chs)
        for nch in range(num_chs):
            
            lfp_this = lfp[trial_idx, nch]
            tax = lfp_this[lfpref_idx['t']].squeeze()
            lfp_val = lfp_this[lfpref_idx['lfp']].squeeze()
            
            dt = np.median(np.diff(tax))
            thetaband = np.array([5, 12]) * dt * 2
            deltaband = np.array([1, 4]) * dt * 2
            B, A = butter(4, thetaband, btype='band')
            Bd, Ad = butter(2, deltaband, btype='band')
            theta_filt = filtfilt(B, A, lfp_val)
            delta_filt = filtfilt(Bd, Ad, lfp_val)
            thetadeltaratio[nch] = np.sum(theta_filt ** 2) / np.sum(delta_filt ** 2)
            
        maxchid = np.argmax(thetadeltaratio)
        lfp_selected = lfp[trial_idx, maxchid]
        lfp_val = lfp_selected[lfpref_idx['lfp']].squeeze()
        tax = lfp_selected[lfpref_idx['t']].squeeze()
        theta_selected = filtfilt(B, A, lfp_val)
        phase = np.angle(hilbert(theta_selected))
        wave_dict = dict()
        wave_dict['theta'] = theta_selected
        wave_dict['lfp'] = lfp_val
        wave_dict['tax'] = tax
        wave_dict['phase'] = phase
        wavedf = pd.DataFrame(wave_dict)
        
        df_dict['animal'].append(animal)
        df_dict['date'].append(date)
        df_dict['behavior'].append(ctxid['ctxID'][trial_idx, 0].item())
        df_dict['trial'].append(trial_idx)
        df_dict['region'].append(region)
        df_dict['position'].append(indata_df)
        df_dict['spikes'].append(spike_df)
        df_dict['wave'].append(wavedf)
dftemp2 = pd.DataFrame(df_dict)





 22/58

/home/yiu/programs/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:62: RuntimeWarning: overflow encountered in square


 57/58

In [366]:
# Merge rows of CA1, CA2 and CA3

sepdf_dict = dict(
    animal=[],
    date=[],
    behavior=[],
    trial=[],
    posdf=[],  # indata for CA1, CA2, CA3 are the same. Merge here.
    CA1_spdf=[],
    CA2_spdf=[],
    CA3_spdf=[],
    wave = []  # wave for CA1, CA2, CA3 are the same. Merge here.
)
getlen = lambda x : len(x.shape)

for (animal, date, trial), gpdf in dftemp2.groupby(by=['animal', 'date', 'trial']):
    behavior = gpdf.iloc[0]['behavior']
    
    sepdf_dict['animal'].append(animal)
    sepdf_dict['date'].append(date)
    sepdf_dict['behavior'].append(behavior)
    sepdf_dict['trial'].append(trial)
    positions_list = []
    waves_list = []
    for ca in ['CA1', 'CA2', 'CA3']:
        
        cadf = gpdf[gpdf['region'] == ca]
        if cadf.shape[0] <1:
            sepdf_dict['%s_spdf'%ca].append(pd.DataFrame(dict(tsp=[])))
        else:
            position = cadf.iloc[0]['position']
            wave = cadf.iloc[0]['wave']
            spikes = cadf.iloc[0]['spikes']
            
            # Clean Spike data
            tsplen = spikes.tsp.apply(getlen)
            spikes.drop(index=spikes[tsplen == 2].index, inplace=True)
            spikes.reset_index(drop=True, inplace=True)
            tsplen = spikes.tsp.apply(getlen)
            spikes.loc[tsplen==0, 'tsp'] = spikes.loc[tsplen==0, 'tsp'].apply(np.atleast_1d)
            
            sepdf_dict['%s_spdf'%ca].append(spikes)
            positions_list.append(position)
            waves_list.append(wave)
            
    sepdf_dict['posdf'].append(positions_list[0])  # doesn't matter CA1/CA2/CA3
    sepdf_dict['wave'].append(waves_list[0])  # doesn't matter CA1/CA2/CA3. 
            
df = pd.DataFrame(sepdf_dict)
df.to_pickle('data/emankin_pythonRaw.pickle')

# Compute occupancy and rate map

In [ ]:
df = pd.read_pickle('data/emankin_pythonRaw.pickle')

In [ ]:
area_thresh = 25
print('Compute occupancy, rate and fields')
# # Compute occupancy
# It takes ~4 hours. Recommend saving the result as checkpoint.
num_sess = df.shape[0]
occ_list = []

CA_fielddf_list = dict(CA1=[], CA2=[], CA3=[])
CA_pairdf_list = dict(CA1=[], CA2=[], CA3=[])
for nsess in range(num_sess):
    print("%d/%d Session - Computing occupancy and rate" % (nsess, num_sess))

    # Occupancy
    posdf = df.loc[nsess, 'posdf']
    x, y, t = posdf.x.to_numpy(), posdf.y.to_numpy(), posdf.t.to_numpy()
    xx, yy, occupancy = get_occupancy(x, y, t)
    occ_list.append(dict(xx=xx, yy=yy, occ=occupancy))
    xbound = (0, xx.shape[1]-1)
    ybound = (0, yy.shape[0]-1)
    x_ax, y_ax = xx[0, :], yy[:, 0]


    # Rate
    trange = (t.max(), t.min())
    x_interp = interp1d(t, x)
    y_interp = interp1d(t, y)



    for ca in ['CA1', 'CA2', 'CA3']:
        pf_list = []
        pairdf_dict = dict(field1_id=[], field2_id=[])
        fielddf_dict = dict(cellid=[], mask=[], xyval=[])
        spdf = df.loc[nsess, '%s_spdf' % (ca)]
        if spdf.shape[0] < 1:
            CA_fielddf_list[ca].append(pd.DataFrame(fielddf_dict))
            CA_pairdf_list[ca].append(pd.DataFrame(pairdf_dict))
            spdf['pf'] = pf_list
            continue
        num_cells = spdf.shape[0]
        for ncell in range(num_cells):

            # Computation of rate map
            tsp = spdf.loc[ncell, 'tsp']
            tsp_in = tsp[(tsp < trange[0]) & (tsp > trange[1])]
            xsp_in = x_interp(tsp_in)
            ysp_in = y_interp(tsp_in)
            freq, rate = placefield(xx, yy, occupancy, xsp_in, ysp_in, tsp_in)
            pf_list.append(dict(freq=freq, rate=rate))

            # Segment placefields
            for mask, xyval in segment_fields(xx, yy, freq, rate, area_thresh):
                fielddf_dict['cellid'].append(ncell)
                fielddf_dict['mask'].append(mask)
                fielddf_dict['xyval'].append(xyval)
        fielddf = pd.DataFrame(fielddf_dict)
        CA_fielddf_list[ca].append(fielddf)
        spdf['pf'] = pf_list


        # Identify pairs
        num_fields = fielddf.shape[0]
        if num_fields > 1:
            for fieldid1, fieldid2 in identify_pairs(fielddf, spdf, x_interp, y_interp, x_ax, y_ax, trange):
                pairdf_dict['field1_id'].append(fieldid1)
                pairdf_dict['field2_id'].append(fieldid2)
        pairdf = pd.DataFrame(pairdf_dict)
        CA_pairdf_list[ca].append(pairdf)


df['occ_dict'] = occ_list
df['CA1fields'] = CA_fielddf_list['CA1']
df['CA2fields'] = CA_fielddf_list['CA2']
df['CA3fields'] = CA_fielddf_list['CA3']
df['CA1pairs'] = CA_pairdf_list['CA1']
df['CA2pairs'] = CA_pairdf_list['CA2']
df['CA3pairs'] = CA_pairdf_list['CA3']
df.to_pickle('data/emankin_pythonProcessed.pickle')

# Check Precession

In [1]:
import os
from os.path import join

import matplotlib.pyplot as plt
import matplotlib as mpl

import numpy as np
import pandas as pd
from astropy.stats import kuiper_two
from matplotlib import cm
import matplotlib.colors as mcol
from pycircstat.descriptive import cdiff
from pycircstat.descriptive import mean as circmean
from pycircstat.tests import vtest, watson_williams, kuiper
from scipy.stats import spearmanr, binom_test, chi2_contingency, chisquare, linregress, pearsonr
import statsmodels.api as smapi
from statsmodels.formula.api import ols
from common.script_wrappers import permutation_test_average_slopeoffset, combined_average_curve, compute_precessangle, \
    permutation_test_arithmetic_average_slopeoffset
from common.comput_utils import midedges, circular_density_1d, linear_circular_gauss_density, \
    unfold_binning_2d, repeat_arr, \
    circ_ktest, shiftcyc_full2half, ci_vonmise, fisherexact, ranksums
from common.linear_circular_r import rcc
from common.utils import load_pickle, sigtext, stat_record, p2str
from common.visualization import plot_marginal_slices, SqueezedNorm, customlegend

from common.shared_vars import fontsize, ticksize, legendsize, titlesize, ca_c, dpi, total_figw


In [2]:
# df = pd.read_pickle('data/emankin_pythonProcessed.pickle')
df = pd.read_pickle('results/emankin_python/single_field.pickle')
# df_ori = pd.read_pickle('results/exp/single_field/singlefield_df.pickle')

In [4]:
precess_df

,x,y,t,v,angle,tsp,spikex,spikey,spikeangle,straightrank,...,rcc_p,wave_t,wave_phase,wave_theta,wave_totalcycles,wave_truecycles,wave_maxperiod,cycfrac,fitted,precess_exist
0,"[-17.080368274971796, -17.243711809042125, -17...","[29.13198450176906, 29.88198388357053, 30.6314...","[66742.787626, 66742.822275, 66742.855414, 667...","[22.153041469821954, 22.97437271600619, 20.963...","[1.785239170518291, 1.7479449654954313, 1.8121...","[66743.1244, 66743.1404, 66743.1462, 66743.297...","[-17.32726235678879, -17.178638905744847, -17....","[35.20276814709461, 35.3700528211356, 35.43069...","[0.8291487310916178, 0.7309432154096324, 0.695...",34.020509,...,0.062773,"[66742.78729062695, 66742.78778214843, 66742.7...","[1.558803155536464, 1.5860362234808503, 1.6132...","[0.004831744409619185, -0.006136300862765199, ...",15.0,5.0,0.144998,0.333333,True,False
1,"[-15.896262541444292, -15.746526708336033, -15...","[30.196759002891856, 31.059737730694515, 32.04...","[66778.223412, 66778.256965, 66778.2904, 66778...","[26.1041566348759, 30.132704718127062, 23.7662...","[1.398996306611722, 1.3657087896600544, 1.5001...","[66778.3534, 66778.3576, 66778.3924, 66778.399...","[-15.444900089816977, -15.435592477565857, -15...","[33.66095434740038, 33.76964799510478, 34.5710...","[1.4523178950331896, 1.4481377445514672, 1.440...",11.640375,...,0.130381,"[66778.22293351953, 66778.22342503906, 66778.2...","[-0.2374353875784895, -0.21144197498226552, -0...","[0.4149885267118261, 0.41712551354295524, 0.41...",5.0,3.0,0.116491,0.600000,True,False
2,"[6.448799369683195, 7.1632860524428805, 7.8777...","[26.726798336219154, 27.622103818340182, 28.51...","[66784.462534, 66784.496186, 66784.529562, 667...","[34.0381921001072, 34.31966804780295, 34.98539...","[0.8972536314135567, 0.8972536314135768, 0.914...","[66784.572, 66784.576, 66784.5884]","[8.791305929228255, 8.8799584141388, 9.1547811...","[29.709501369238016, 29.827530272174958, 30.19...","[0.9329612061862137, 0.9358077803065508, 0.944...",15.643698,...,0.157299,"[66784.46228844921, 66784.46277996874, 66784.4...","[-2.0659629196269913, -2.038784615220424, -2.0...","[-0.27723202301227207, -0.26341045603844665, -...",5.0,1.0,0.134185,0.200000,True,False
3,"[17.055218583412394, 16.300713407214484, 15.45...","[37.73946313238875, 38.01388176584818, 38.2250...","[66790.334161, 66790.358521, 66790.400989, 667...","[32.95811447601104, 20.464903240142373, 27.291...","[2.792759476028038, 2.8961298592187323, 2.8340...","[66790.5881, 66790.594, 66790.6025, 66790.6096...","[10.378147115047666, 10.22452861665106, 10.003...","[38.85433139680737, 38.845930255905515, 38.833...","[-3.082209978866619, -3.0807733229697045, -3.0...",37.780712,...,0.009186,"[66790.33398676953, 66790.33447828906, 66790.3...","[-2.947394209794723, -2.9215515993822896, -2.8...","[-0.3447311231534929, -0.3428261158072933, -0....",14.0,7.0,0.142049,0.500000,True,True
4,"[-2.974252456086689, -1.9848842712564871, -0.9...","[26.033085720279626, 26.77693457883458, 27.585...","[66795.140767, 66795.161667, 66795.206617, 667...","[59.225106342684036, 28.694466431562205, 38.42...","[0.6446791429834322, 0.67746979818397, 0.65495...","[66795.4299, 66795.4344, 66795.4415]","[4.89934611521687, 5.000002482622799, 5.158558...","[32.37140509069136, 32.46653894028763, 32.6161...","[0.7364493666375554, 0.7321437976053432, 0.725...",20.789654,...,0.157299,"[66795.14056026562, 66795.14105178515, 66795.1...","[1.107280516114437, 1.1337310329236199, 1.1601...","[0.25085462431889494, 0.23770717367862454, 0.2...",7.0,1.0,0.127795,0.142857,True,False
5,"[-24.214193946181986, -23.45408678655686, -22....","[35.764674568995794, 35.84732537805802, 35.960...","[66806.417962, 66806.451509, 66806.485052, 668...","[22.791531386368874, 25.510353848712676, 29.21...","[0.10831020337307466, 0.1329290202845398, 0.06...","[66806.5449, 66806.5515, 66806.7987, 66806.804...","[-20.800113678548342, -20.593989697152832, -13...","[36.033905902192465, 36.035312321371265, 35.99...","[-0.0

In [6]:
print(df.columns)
print(precess_df.columns)

Index(['ca', 'num_spikes', 'border', 'aver_rate', 'peak_rate', 'rate_angle',
       'rate_R', 'rate_R_pval', 'field_area', 'field_bound', 'precess_df',
       'precess_angle', 'precess_angle_low', 'precess_R', 'precess_R_pval',
       'numpass_at_precess', 'numpass_at_precess_low', 'ratemap',
       'fieldid_ca'],
      dtype='object')
Index(['x', 'y', 't', 'v', 'angle', 'tsp', 'spikex', 'spikey', 'spikeangle',
       'straightrank', 'chunked', 'rejected', 'excluded_for_precess', 'dsp',
       'pass_nspikes', 'phasesp', 'tsp_withtheta', 'mean_angle',
       'mean_anglesp', 'rcc_m', 'rcc_c', 'rcc_rho', 'rcc_p', 'wave_t',
       'wave_phase', 'wave_theta', 'wave_totalcycles', 'wave_truecycles',
       'wave_maxperiod', 'cycfrac', 'fitted', 'precess_exist'],
      dtype='object')


In [38]:

adiff_ca = dict()
offsets_ca = dict()
slopes_ca = dict()

# for ca, cadf in df.groupby('ca'):
ca = 'CA3'
# cadf = cadf[cadf['numpass_at_precess']>0].reset_index(drop=True)
cadf = cadf.reset_index(drop=True)
rateangle_list, passangle_list, offset_list, slope_list = [], [], [], []

for i in range(cadf.shape[0]):

    rate_angle, pass_df = cadf.loc[i, ['rate_angle', 'precess_df']]

    precess_df = pass_df[pass_df['precess_exist']]
    num_precess = precess_df.shape[0]
    passangles, offset, slope = precess_df['mean_anglesp'].to_numpy(), precess_df['rcc_c'].to_numpy(), precess_df['rcc_m'].to_numpy()


    rateangle_list.extend([rate_angle]*num_precess)
    passangle_list.extend(passangles)
    offset_list.extend(offset)
    slope_list.extend(slope)

rateangles = np.array(rateangle_list)
passangles = np.array(passangle_list)
offsets = np.array(offset_list)
slopes = np.array(slope_list)*2*np.pi

adiff = np.abs(cdiff(rateangles, passangles))

frac = 1/6
highmask = adiff > (np.pi - (np.pi*frac))
lowmask = adiff < (np.pi*frac)

highn, lown = highmask.sum(), lowmask.sum()
print('%s high(n=%d), low(n=%d)'%(ca, highn, lown))
highsam = min(highn, 500)
lowsam = min(lown, 500)
np.random.seed(1)
high_ranvec = np.random.choice(highn, highsam, replace=False)
np.random.seed(2)
low_ranvec = np.random.choice(lown, highsam, replace=False)

highoffsets, lowoffsets = offsets[highmask][high_ranvec], offsets[lowmask][low_ranvec]
highslopes, lowslopes = slopes[highmask][high_ranvec], slopes[lowmask][low_ranvec]



p_offset_hl, _ = watson_williams(highoffsets, lowoffsets)
_, p_slope_hl = ranksums(highslopes, lowslopes)

print('Onset: High(%0.2f) vs Low(%0.2f), p%s'%(circmean(highoffsets), circmean(lowoffsets), p2str(p_offset_hl)))
print('Slope: High(%0.2f) vs Low(%0.2f), p%s'%(np.median(highslopes), np.median(lowslopes), p2str(p_slope_hl)))


regress_high, regress_low, pval_slope, pval_offset = permutation_test_average_slopeoffset(highslopes, highoffsets, lowslopes, lowoffsets, NShuffles=100)
print('Permutation test p_slope%s, p_offset%s'%(p2str(pval_slope), p2str(pval_offset)))
    

CA3 high(n=496), low(n=624)
Onset: High(3.76) vs Low(3.75), p=0.9171
Slope: High(-3.58) vs Low(-3.80), p=0.1532
Shuffling 0/100
Shuffling 10/100
Shuffling 20/100
Shuffling 30/100
Shuffling 40/100
Shuffling 50/100
Shuffling 60/100
Shuffling 70/100
Shuffling 80/100
Shuffling 90/100
Permutation test p_slope=0.2000, p_offset=0.6000
